In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
from data.animals10 import Animals10
from torchvision import transforms

dataset = Animals10(src='/home/kurikuri/Projects/Datasets/Animals-10/',
                    root='../dataset/animals10',
                    prepare=False,
                    transform=(
                        transforms.ToTensor(),
                        transforms.Resize((448, 448))
                    ))